In [33]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

In [2]:
kml_file = 'https://dl.dropbox.com/s/jbzpfuc29fab3xd/anzacs.kml?dl=0'
response = requests.get(kml_file)

In [3]:
response.status_code

200

In [17]:
soup = BeautifulSoup(response.text)
placemarks = soup.find_all('placemark')

In [47]:
places = []
for pm in placemarks:
    name = pm.find('name').string
    coords = pm.point.coordinates.string
    longitude, latitude, _ = coords.split(',')
    details = list(pm.description.stripped_strings)
    for i, d in enumerate(details):
        if d.startswith('Born'):
            born = int(details[i+1])
        elif d.startswith('Enlisted'):
            enlisted = int(details[i+1])
    link = pm.description.find('a')
    try:
        fullname = re.search(r'=(.+$)', link['href']).group(1)
    except TypeError:
        pass
    else:
        place_parts = fullname.split(', ')
        if len(place_parts) == 3:
            state = place_parts[1]
            country = place_parts[2]
        elif len(place_parts) == 2:
            state = ''
            country = place_parts[1]
        elif len(place_parts) == 1:
            state = ''
            country = place_parts[0]
        place = {
            'name': name,
            'display_name': fullname,
            'state': state,
            'country': country,
            'born': born,
            'enlisted': enlisted,
            'latitude': latitude,
            'longitude': longitude
        }
        places.append(place)

In [48]:
df = pd.DataFrame(places)

In [49]:
df.to_csv('b2455_places.csv', index=False)